上次批量提取了上市公司主要业务信息，要分析这些文本数据，就需要做文本词频分析。由于中文不同于英文，词是由一个一个汉字组成的，而英文的词与词之间本身就有空格，所以中文的分词需要单独的库才能够实现，常用的是`jieba`。若没安装，直接运行`cmd`，然后输入`pip install jieba`安装即可。然后导入`jieba`库。我们以“华特气体”公司的主要业务进行分词，分词前如下图所示。通过`open`打开华特气体文本文件，使用读模式`r`，为避免编码错误，指定编码类型为`utf-8`。读取出来是一个大字符串，将这个大字符串存入`txt`。然后调用`jieba`进行分词。`lcut`的意思是切分词后再转换成列表("l"即表示`list`的首字母)。将切分后的词存入列表`words`。
![](images\before.png)

In [90]:
import jieba
txt = open("华特气体.txt", "r", encoding="utf-8").read()
words = jieba.lcut(txt)
words

['主要',
 '业务',
 '\\',
 '华特',
 '气体',
 '：',
 '2019',
 '年',
 '年度报告',
 '（',
 '修订版',
 '）',
 '\n',
 '一',
 '、',
 ' ',
 '报告',
 '期内',
 '公司',
 '所',
 '从事',
 '的',
 '主要',
 '业务',
 '、',
 '经营',
 '模式',
 '、',
 '行业',
 '情况',
 '及',
 '研发',
 '情况',
 '说明',
 ' ',
 '\n',
 '(',
 '一',
 ')',
 ' ',
 '主要',
 '业务',
 '、',
 '主要',
 '产品',
 '或',
 '服务',
 '情况',
 ' ',
 '\n',
 '1',
 '、',
 '主营业务',
 '基本',
 '情况',
 ' ',
 '\n',
 '报告',
 '期内',
 '公司',
 '主营业务',
 '以',
 '特种',
 '气体',
 '的',
 '研发',
 '、',
 '生产',
 '及',
 '销售',
 '为',
 '核心',
 '，',
 '辅以',
 '普通',
 '工业气体',
 '、',
 '气体设备',
 '与',
 '\n',
 '工程',
 '业务',
 '，',
 '打造',
 '一站式',
 '服务',
 '能力',
 '，',
 '能够',
 '面向',
 '全球',
 '市场',
 '提供',
 '气体',
 '应用',
 '综合',
 '解决方案',
 '。',
 ' ',
 '\n',
 '10',
 ' ',
 '/',
 ' ',
 '212',
 ' ',
 '\n',
 ' ',
 '2019',
 '年',
 '年度报告',
 ' ',
 '\n',
 '2',
 '、',
 '公司',
 '提供',
 '的',
 '主要',
 '产品',
 '和',
 '服务',
 '情况',
 '如下',
 '：',
 ' ',
 '\n',
 '类别',
 ' ',
 ' ',
 '简介',
 '及',
 '用途',
 ' ',
 '\n',
 '特种',
 '气体',
 ' ',
 ' ',
 '1',
 '、',
 '广泛',
 '用于',
 '集成电路',
 '、',
 '新型',
 '显示',
 '面板'

结果如上，可见基本是按照我们的汉字使用习惯来区分词的，不能组成词的字则是按单独一个字符串存放的。然后我们就需要将词和对应出现的次数统计出来。先新建一个字典`wordsDict`用于储存词及其出现的次数。对于单个的字或字符不是我们想要的，所以加了一个`if`语句将其排除在外。`continue`的作用是，`if`后面的条件满足时，让程序回到`for`循环，而不执行`continue`下面的语句。也就是列表中的元素只有一个字符的时候，就马上跳到下一个而不执行任何操作。只有当元素不止一个字符的时候，才执行`else`语句，即将词及其出现的次数加入字典。此处用`setdefault`给词的出现初始值设置为0，每重复出现一次，自动加1。然后我们根据此出现的次数，降序排序，并查看前20个词的情况。

In [91]:
wordsDict = {} #新建字典用于储存词及词频
for word in words:
    if len(word) == 1: #单个的字符不作为词放入字典
        continue
    else:
        wordsDict.setdefault(word, 0) #设置词的初始出现次数为0
        wordsDict[word] +=1 #对于重复出现的词，每出现一次，次数增加1
        
wordsDict_seq = sorted(wordsDict.items(),key=lambda x:x[1], reverse=True) #按字典的值降序排序
wordsDict_seq[:15] 

[('气体', 151),
 ('公司', 76),
 ('特种', 52),
 ('行业', 42),
 ('产品', 42),
 ('高纯', 42),
 ('生产', 41),
 ('000', 35),
 ('客户', 34),
 ('发展', 34),
 ('研发', 31),
 ('产业', 30),
 ('技术', 30),
 ('主要', 29),
 ('实验', 24)]

可以看到，有些词并不是我们想要的，比如“公司”、“行业”、“000”。因此需要把这些意义不大的词删除。先定义一个储存要排除的词的列表`stopWords`，将想排除的词放进去，一般是放出现次数较多，但意义不大的词，可根据实际需要调整。然后遍历这个字典，在检查这些词是否在目标字典`wordsDict`中，如果在，就将字典中这个词对应的数据删除。

In [92]:
stopWords = ["公司","行业","000","用于","情况","方面","一种","要求","对于","进行","一般","212","实现","处理","通过","投入","随着"]
for word in stopWords:
    if word in wordsDict:
        del wordsDict[word] #删除对应的词
        
wordsDict_seq = sorted(wordsDict.items(),key=lambda x:x[1], reverse=True) #按字典的值降序排序
wordsDict_seq[:15] 

[('气体', 151),
 ('特种', 52),
 ('产品', 42),
 ('高纯', 42),
 ('生产', 41),
 ('客户', 34),
 ('发展', 34),
 ('研发', 31),
 ('产业', 30),
 ('技术', 30),
 ('主要', 29),
 ('实验', 24),
 ('国内', 23),
 ('2019', 22),
 ('模式', 22)]

然后将筛选后的数据转换成DataFrame，并增加列名“词”和“次数”，然后导出为Excel文件。

In [95]:
df = pd.DataFrame(wordsDict_seq,columns=['词','次数'])
df.to_excel("华特气体-词频.xlsx",index = False) #存为Excel时去掉index索引列
df.head(10)

,词,次数
0,气体,151
1,特种,52
2,产品,42
3,高纯,42
4,生产,41
5,客户,34
6,发展,34
7,研发,31
8,产业,30
9,技术,30


以上，搞定了一个文件的词频收集，那批量操作呢？请看下面分解。

In [96]:
import os
path='主要业务'  #文件所在文件夹
files = [path+"\\"+i for i in os.listdir(path)] #获取文件夹下的文件名,并拼接完整路径
files

['主要业务\\东旭蓝天：2019年年度报告.txt',
 '主要业务\\保利地产：2018年年度报告.txt',
 '主要业务\\共达电声：2019年年度报告（更新后）.txt',
 '主要业务\\华特气体：2019年年度报告（修订版）.txt',
 '主要业务\\吉峰科技：2019年年度报告（更新后）.txt',
 '主要业务\\引力传媒：2019年年度报告（修订版）.txt',
 '主要业务\\方正科技：2019年年度报告.txt',
 '主要业务\\湖北宜化：2019年年度报告（更新后）.txt',
 '主要业务\\联创股份：2019年年度报告（更新后）.txt',
 '主要业务\\高乐股份：2019年年度报告.txt']

以上，先获取到所有待分析文件的路径。然后逐个进行分析。稍微修改一下上面的程序，很快分析完成。结果如下。
![](images\\result.png)

In [ ]:
import jieba
import pandas as pd

for file in files:

    txt = open(file, "r", encoding="utf-8").read()
    words = jieba.lcut(txt)
    wordsDict = {} #新建字典用于储存词及词频
    for word in words:
        if len(word) == 1: #单个的字符不作为词放入字典
            continue
        else:
            wordsDict.setdefault(word, 0) #设置词的初始出现次数为0
            wordsDict[word] +=1 #对于重复出现的词，每出现一次，次数增加1

    stopWords = ["2019","不断","持续","主要","企业","产品","业务","公司","行业","000","用于","情况","方面","一种","要求","对于","进行","一般","212","实现","处理","通过","投入","随着"]
    for word in stopWords:
        if word in wordsDict:
            del wordsDict[word] #删除对应的词

    wordsDict_seq = sorted(wordsDict.items(),key=lambda x:x[1], reverse=True) #按字典的值降序排序

    df = pd.DataFrame(wordsDict_seq,columns=['词','次数'])
    df.to_excel("词频//{}.xlsx".format(file.split("\\")[1][:-4]),index = False) #存为Excel时去掉index索引列